# Homework

## Questions

### Question 1. Embedding the query

In [1]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models
import numpy as np

In [2]:
homework_client = QdrantClient("http://localhost:6333")

Run the qdrant container in a new bash window and open the dashboard in the browser: http://localhost:6333/dashboard

In [ ]:
!docker run -p 6333:6333 qdrant/qdrant

In [11]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = 'jinaai/jina-embeddings-v2-small-en'
embedder = TextEmbedding(model_name = model_handle)

In [12]:
query = 'I just discovered the course. Can I join now?'
query_vector = list(embedder.embed([query]))[0]
print(len(query_vector))

512


In [13]:
query_vector_np = np.array(query_vector)
min_value = np.min(query_vector_np)

print(f"Minimum value in the embedding vector: {min_value:.3f}")

Minimum value in the embedding vector: -0.117


In [14]:
print(np.linalg.norm(query_vector))

1.0


In [15]:
print(query_vector.dot(query_vector))

1.0000000000000002


### Question 2. Cosine similarity with another vector

In [16]:
doc = 'Can I still join the course after the start date?'
doc_vector = list(embedder.embed([doc]))[0]

In [17]:
# print(query_vector.dot(doc_vector))
# print(doc_vector.dot(query_vector))
print(np.dot(query_vector, doc_vector))

0.9008528895674548


### Question 3. Ranking by cosine

In [23]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [20]:
texts = []
texts = [doc['text'] for doc in documents]
documents_vectors = list(embedder.embed(texts))

# Stack into a 2D array
V = np.vstack(documents_vectors)

V.dot(query_vector)

array([0.76296845, 0.81823782, 0.80853974, 0.71330788, 0.73044992])

In [21]:
cosine_results = []

for i, doc in enumerate(documents):
    text = doc['text']
    text_vector = list(embedder.embed([text]))[0]
    similarity = text_vector.dot(query_vector)
    cosine_results.append(similarity)
    print(f"Document {i} has {similarity} similarity with our query.")

Document 0 has 0.762968451872193 similarity with our query.
Document 1 has 0.8182378156620136 similarity with our query.
Document 2 has 0.8085397445747489 similarity with our query.
Document 3 has 0.7133078832064158 similarity with our query.
Document 4 has 0.7304499196411823 similarity with our query.


### Question 4. Ranking by cosine, version two

In [24]:
text = []
combined_similarity = []
question_tree = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    combined_vector = list(embedder.embed([text]))[0]
    combined_similarity = combined_vector.dot(query_vector)
    question_tree.append(combined_similarity)
    print(question_tree[i])
    

0.851454319443226
0.8436594005975433
0.8408287224005012
0.7755157657626952
0.8086007917931166


In [25]:
# Combine 'question' and 'text' for each document
texts = []
texts = [f"{doc['question']} {doc['text']}" for doc in documents]

# Embed the combined texts
documents_vectors = []
documents_vectors = list(embedder.embed(texts))

# Stack into a 2D array
V = np.vstack(documents_vectors)

# Compute cosine similarity with the query vector
similarities = V.dot(query_vector)
similarities

array([0.85145432, 0.8436594 , 0.84082872, 0.77551577, 0.80860079])

### Question 5. Selecting the embedding model

In [26]:
TextEmbedding.list_supported_models()

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [ ]:
# for model in TextEmbedding.list_supported_models():
#     print(model["dim"])

model_dimensions = []
for i, model in enumerate(TextEmbedding.list_supported_models()):
    model_dimensions.append(model["dim"])
    print(model_dimensions[i])

In [33]:
print(f"The smallest dimensionality for models in fastembed is {np.min(model_dimensions)}.")

The smallest dimensionality for models in fastembed is 384.


### Question 6. Indexing with qdrant

In [35]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
EMBEDDING_DIMENSIONALITY = 384
model_handle = 'BAAI/bge-small-en'
embedder = TextEmbedding(model_name = model_handle)

In [39]:
collection_name = "homework-rag"

In [49]:
homework_client.delete_collection(collection_name=collection_name)

True

In [50]:
homework_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [45]:
homework_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [46]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [47]:
points[0]

PointStruct(id=0, vector=Document(text='How do I sign up? Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', model='BAAI/bge-small-en', options=None), payload={'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', 'section': 'General course-related questions', 'question': 'How do I s

In [51]:
homework_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [53]:
question = 'I just discovered the course. Can I join now?'

In [72]:
def vector_search(question, limit):
    
    query_points = homework_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        limit=limit,
        with_payload=True
    )

    return query_points.points

In [73]:
result = vector_search(question, limit = 1)

In [78]:
print(f"The score of the first returned result is {result[0].score}.")

The score of the first returned result is 0.87031734.
